### I have a dataset of neural activities with small sample size and high number of neurons. I want to find the subspaces encoding different task conditions: the stimuli A vs B and the choice of the animal Lick/ no Lick. How can I proceed?

Here’s one way to approach the problem:

1.  Define your goals clearly. You want to extract subspaces (or latent
    low‐dimensional representations) that are associated with (a) the
    stimulus (A vs. B) and (b) the behavioral choice (Lick vs. no Lick).

2.  Preprocess the data. Because you have many neurons relative to the
    number of trials, consider:

 • Normalizing the firing rates (or z‐scaling) per neuron.  • Removing
(or regressing out) any common baseline activity if needed.  • Possibly
averaging across similar trials to improve the signal/noise ratio.

1.  Dimensionality reduction as a first step. The high dimensionality
    (many neurons) and small sample size can lead to overfitting so a
    dimensionality reduction method is advisable:

 • Start with a method like Principal Component Analysis (PCA) to
capture the major sources of variance. Although PCA itself is “blind” to
task variables, it can reduce noise and reduce dimensions from the
many-neuron space to a few dozen “components.”

1.  Use demixed analysis techniques. To tease apart the contributions of
    stimulus and choice, consider “demixed” methods such as demixed
    Principal Component Analysis (dPCA). dPCA is specifically designed
    for neural data where different task parameters (e.g., stimulus
    identity, decision/choice) are intermingled in the population
    activity. It finds projections that maximize explained variance
    while “demixing” the variance attributable to different task
    parameters. Steps involve:

 • Labeling each trial according to its task conditions (stimulus A vs.
B, and Lick vs. no Lick).  • Running dPCA on your trial data to obtain
components that are predominantly tuned to stimulus, mostly to choice,
and possibly to interaction effects.  • Inspecting the resulting
components/subspaces to see if they align with your expectations (e.g.,
variance explained, separation between conditions).

1.  Alternative/Complementary methods:

 • Linear Discriminant Analysis (LDA) or logistic regression with
regularization: These methods can help in finding directions that best
discriminate between conditions (A vs. B, Lick vs. no Lick). However,
because of the small sample size, you’d need to use regularization
(L1/L2) or even shrinkage methods.  • Targeted dimensionality reduction
procedures, where you first build a classifier (again taking care with
regularization and cross-validation) and then inspect the classifier
weights to see which neurons or neural directions are most informative.

1.  Cross-validation and robustness. Given the small sample size you
    mentioned, it’s critical to:

 • Use cross-validation (or bootstrapping) to ensure that the identified
subspaces generalize.  • Possibly use permutation testing to establish
the significance of the separations (i.e., that the subspace dimensions
are not just overfitting noise).

1.  Interpret the subspaces. Once you have obtained the subspaces:

 • Visualize the projections. For example, plot the trials in the
low-dimensional space and see if the stimulus and choice conditions
segregate.  • Quantify how much of the variance (and specifically,
task-related variance) is captured in each subspace.  • Compare these
findings to your theoretical expectations about how the encoding might
be organized.

In summary, a promising pipeline is to first apply PCA (or another
noise-reduction method) and then employ a demixed analysis like dPCA,
which will allow you to separate the contributions from the stimulus and
the choice despite having a high-dimensional but limited-sample dataset.
This approach has been successfully applied in many neural data analyses
(see e.g., Kobak et al.’s work on dPCA).

By combining these methods with careful cross-validation and
regularization strategies to guard against overfitting, you should be
able to extract meaningful subspaces corresponding to the different task
conditions.

### How would you preprocess the data? the data is fluorescence signals from neurons in the shape (n<sub>samples</sub>, n<sub>neurons</sub>, n<sub>time</sub>).

Here’s a step-by-step guide on how you might preprocess your
fluorescence signal data with shape (n<sub>samples</sub>,
n<sub>neurons</sub>, n<sub>time</sub>):

1.  Align Trials and Define Epochs:

 • Ensure that your time axis is correctly aligned to relevant events
(e.g., stimulus onset, response, etc.).  • Define baseline, stimulus,
and response epochs so you can later compute baseline corrections.

1.  Baseline Correction (dF/F Calculation):

 • For each trial and each neuron, identify a baseline period (often a
time window before stimulus onset).  • Compute the baseline fluorescence
(F₀), for example by taking the average fluorescence over the baseline
period.  • Calculate the relative change in fluorescence as dF/F:
   dF/F = (F - F₀) / F₀   This normalization minimizes differences due
to varying absolute fluorescence levels across neurons and trials.

1.  Denoising and Smoothing:

 • Fluorescence signals tend to be noisy. Apply a temporal filter (e.g.,
a low-pass filter or a moving average) to smooth the signals.  •
Alternatively, you can use more advanced denoising algorithms (e.g.,
wavelet denoising or PCA-based denoising) if the noise is substantial.
 • Be cautious—not to oversmooth—to preserve the temporal dynamics
critical for task-related activity.

1.  Normalization Across Trials and Neurons:

 • After computing dF/F, it can be useful to standardize the data. You
might z-score the data for each neuron across all time points and
trials: subtract the mean and divide by the standard deviation.  • This
step helps when neurons have vastly different dynamic ranges, ensuring
that subsequent analyses (like PCA or dPCA) are not dominated by neurons
with inherently larger fluctuations.

What do you mean by across all time points and trials?

When we say "across all time points and trials," we mean that for each
neuron you treat all of its data—over every trial and every time step—as
one pool of observations when calculating its mean and standard
deviation for normalization.

For example, suppose you have a neuron with a data array that has
dimensions corresponding to trials and time (after any initial
processing like dF/F). You would concatenate all of these values (or
equivalently, compute statistics over the entire (n<sub>samples</sub> ×
n<sub>time</sub>) array for that neuron) to get a single mean value and
standard deviation. Then, for each actual time point in each trial, you
subtract that mean and divide by that standard deviation. This results
in a z-scored time course for that neuron which has a mean of 0 and a
standard deviation of 1 based on all available data.

Normalizing in this way helps ensure that neurons with inherently
different scales or levels of variability contribute comparably during
later analyses like PCA or dPCA.

### But if a neuron is stimulus selectivve then it will bias its mean activity no ?

Yes, using all time points to compute the mean can incorporate
stimulus-evoked changes, which might bias the normalization if the
response magnitude is high. This happens because the normalization then
reflects not just the baseline variability but also the task-related
modulation.

One way to avoid this is to compute the baseline mean and standard
deviation using a time window when the stimulus is absent (e.g.,
pre-stimulus period). Then, you can normalize each trial using these
baseline statistics so that the effect of the stimulus doesn't skew the
overall normalization.

### what is best?

It depends on your goals, but a baseline normalization (using a
pre-stimulus time window to compute mean and standard deviation) is
often preferred. This avoids biasing the normalization with
stimulus-evoked activity and better highlights changes relative to a
known “resting” state. However, if you're interested in global activity
patterns across the whole trial, normalizing over all time points may be
acceptable. In practice, baseline normalization is usually considered
best for isolating task-evoked responses from the intrinsic variability
of each neuron.

**\***

1.  Artifact Removal:

 • Inspect the data for any trials or neurons with obvious artifacts
(e.g., sudden jumps or drops in signal not related to neural activity).
 • Consider excluding or correcting these artifacts, as they can bias
downstream dimensionality reduction or decoding analyses.

1.  Trial Averaging (if applicable):

 • If your sample size is extremely limited and if it’s appropriate to
your experimental design, you might average across trials of the same
condition to improve your signal-to-noise ratio.  • However, be careful
as averaging can mask trial-to-trial variability you might later find
informative.

1.  Optional: Additional Filtering or Feature Extraction:

 • You might perform additional filtering (e.g., detrending if there is
drift) or extract features (like peak amplitudes, onset latencies, etc.)
if these features are useful for your analysis.  • Some analyses benefit
from reducing data dimensionality at this stage (e.g., by computing
summary statistics within defined time windows).

1.  Verification:

 • Visualize a few single-trial traces and the averaged responses to
ensure that the preprocessing steps (dF/F and smoothing) have resulted
in meaningful signals.  • Check that the fluctuations correspond to your
expected neural events or task-related responses.

Following these steps should prepare your fluorescence signal data for
further dimensionality reduction and subspace analysis (e.g., PCA, dPCA,
or LDA) aimed at isolating the neural encoding of the task conditions.

**\***

### Are you sure that I should use PCA first?

It's not absolutely mandatory to apply PCA first, but there are reasons
why doing so is often beneficial—especially in your situation with many
neurons and relatively few experimental trials. Here are a few points to
consider:

1.  Noise reduction and regularization:

 • PCA helps you reduce the dimensionality by concentrating on the
directions with the highest variance.  • With a high number of neurons
and limited data, you risk overfitting any method you apply. A PCA-based
reduction can filter out noise, improving the signal-to-noise ratio for
subsequent analyses.

1.  Computational simplicity:

 • Reducing the dimensionality before applying a method like demixed PCA
(dPCA) or LDA can lower computational costs and simplify the model.

1.  Preservation of task-relevant variance:

 • One concern is that if you remove too many dimensions with PCA, you
might discard variance that carries information about your task
conditions (stimulus A vs. B or lick vs. no lick).  • To mitigate this,
you can choose the number of PCA components carefully—for example, by
cross-validating the selection or examining the cumulative variance
explained. If you see that most task-related variance is retained, then
it’s a justified step.

1.  Alternative approaches:

 • There are methods that directly incorporate task labels (like dPCA)
or supervised approaches (like regularized LDA) that can potentially
work without the preliminary PCA step. However, these methods can also
benefit from an initial dimensionality reduction to mitigate
overfitting.

In summary, while you’re not forced to use PCA as a first step, it is a
common and practical method in neuroscience research to handle the curse
of dimensionality and small sample sizes, provided you’re cautious with
how much variance you throw away. If you decide not to use it, or if
you’re worried about losing task-relevant information, you might instead
apply a method like dPCA directly with appropriate regularization and
cross-validation to ensure the robustness of the subspaces you extract.

**\***

Yes, you can use bootstrapping—but there are some nuances to consider
given your setting of high-dimensional neural data with a relatively
small sample size. Here’s a breakdown of how you might proceed and some
caveats:

1.  Bootstrapping for Estimation and Confidence Intervals:

  • You can resample your trials (or task conditions) with replacement
and re-run dPCA on each sample. This allows you to estimate the
variability (or confidence intervals) of the extracted components and
subspaces.   • Doing so can help you gauge which components are robust
versus those that may be artifacts of your limited dataset.

1.  Bootstrapping for Stability Assessment:

  • Beyond confidence intervals, bootstrapping can help check the
stability of the stimulus-encoding subspaces. If, across many
bootstrapped samples, the same subspaces appear, then you may have
greater confidence in your findings.   • Conversely, if the estimated
subspaces vary widely, it may be a sign of overfitting or that your data
are too noisy for the current method.

1.  Considerations:

  • Small Sample Size: Bootstrapping works best when the resampled
distributions are representative of the true underlying distribution;
with very few samples, each bootstrap replicate might be highly
variable. This can lead to uncertainty in your estimates.   •
Overfitting: High-dimensional data (many neurons) might lead to
overfitting in each bootstrap sample. Regularizing your dPCA solution
(or the preprocessing steps) might help mitigate this issue.   •
Alternative Methods: Given the sample size issue, you might also
consider other dimensionality reduction techniques that are robust in
high dimensions, such as regularized methods, factor analysis, or sparse
coding approaches. Bayesian latent variable models can also incorporate
prior information that might help with small sample sizes.

1.  Practical Steps:

  • Resampling: Decide whether to bootstrap across trials, neurons, or
both. Often, trials are the natural resampling unit if the neurons are
recorded simultaneously.   • Cross-Validation: Combine bootstrapping
with cross-validation to help prevent overfitting. For instance, use
bootstrapped samples to determine the optimal model complexity (number
of dPCA components) on a training set, then validate on held-out data.
  • Report Uncertainty: Use the bootstrapped estimates to report the
variability of the projections. This can provide more insight into which
dimensions are statistically robust.

In summary, bootstrapping is a viable tool in your analysis toolbox,
both for estimating the stability of your dPCA results and for assessing
uncertainty in your estimated stimulus-encoding subspaces. Just ensure
that you’re careful in interpreting the results, particularly given the
limitations imposed by small sample sizes in a high-dimensional setting.

**\***